### **Code : Natural Products - Hierarchical Bayesian**
##### Writer : Donghyeon Kim
##### Update : 2025.01.02.

---

#### **0. Package Reference**
```
conda create -c conda-forge -n pymc_env "pymc>=5"
conda activate pymc_env

pip install statsmodels

pip install -U scikit-learn
```

---

#### **1. Prior Settings**

In [1]:
import os

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.formula.api import ols # Regression Model
import pymc as pm # Bayesian Analysis
import arviz as az
from sklearn.cluster import KMeans # K-means Clustering

##### Data : 천연물 본설문 OLS

In [2]:
DATA = pd.read_csv('C:/Users/mazy4/Dropbox/6. C&S Lab/7. 2024 Project/4. 야생생물 경제가치_생물자원관/2. 분석/2. Data/2. 본설문/Naturalproducts_Main_Profile(53).csv')
DATA.head(10)

,INDEX,CARD,BIO1,BIO2,BIO3,SCA1,SCA2,SCA3,KNO1,KNO2,KNO3,USE1,USE2,USE3,MET1,MET2,ANA1,ANA2,PRICE,Y
0,14,1,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,2000000,2
1,14,2,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,1,300000,3
2,14,3,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,1,30000,5
3,14,4,1,0,0,0,1,0,0,0,1,1,0,0,0,1,0,1,2000000,1
4,14,5,0,1,0,0,0,1,0,1,0,1,0,0,0,1,1,0,300000,4
5,14,6,0,0,1,1,0,0,0,0,1,0,0,1,1,0,1,0,300000,2
6,14,7,0,1,0,0,0,1,0,0,1,0,1,0,1,0,1,0,30000,6
7,14,8,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,30000,3
8,14,9,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,30000,6
9,14,10,0,0,1,0,1,0,1,0,0,0,1,0,0,1,1,0,30000,6


In [3]:
# PRICE category = (1, 2, 3)
DATA.loc[(DATA['PRICE'] == 30000),'PRICE'] = 1
DATA.loc[(DATA['PRICE'] == 300000),'PRICE'] = 2
DATA.loc[(DATA['PRICE'] == 2000000),'PRICE'] = 3
DATA.head(20)

,INDEX,CARD,BIO1,BIO2,BIO3,SCA1,SCA2,SCA3,KNO1,KNO2,KNO3,USE1,USE2,USE3,MET1,MET2,ANA1,ANA2,PRICE,Y
0,14,1,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,3,2
1,14,2,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,1,2,3
2,14,3,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,1,1,5
3,14,4,1,0,0,0,1,0,0,0,1,1,0,0,0,1,0,1,3,1
4,14,5,0,1,0,0,0,1,0,1,0,1,0,0,0,1,1,0,2,4
5,14,6,0,0,1,1,0,0,0,0,1,0,0,1,1,0,1,0,2,2
6,14,7,0,1,0,0,0,1,0,0,1,0,1,0,1,0,1,0,1,6
7,14,8,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,3
8,14,9,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,1,6
9,14,10,0,0,1,0,1,0,1,0,0,0,1,0,0,1,1,0,1,6


---

#### **2. 전체 응답자 OLS**

In [4]:
def get_ols_params_table(data_df):
    ols_model = ols("Y ~ BIO2 + BIO3 + SCA2 + SCA3 + KNO2 + KNO3 + USE2 + USE3 + MET2 + ANA2 + PRICE", data=data_df)
    ols_result = ols_model.fit()
    ols_params = pd.DataFrame(ols_result.params, columns=['Coefficient'])
    ols_params['Standard error'] = ols_result.bse
    ols_params['P-value'] = ols_result.pvalues.round(6)
    
    ols_params_2 = ols_params[['Coefficient', 'Standard error']].copy()
    add_params = [
        -ols_params.loc['BIO2',['Coefficient', 'Standard error']] -ols_params.loc['BIO3',['Coefficient', 'Standard error']],
        -ols_params.loc['SCA2',['Coefficient', 'Standard error']] -ols_params.loc['SCA3',['Coefficient', 'Standard error']],
        -ols_params.loc['KNO2',['Coefficient', 'Standard error']] -ols_params.loc['KNO3',['Coefficient', 'Standard error']],
        -ols_params.loc['USE2',['Coefficient', 'Standard error']] -ols_params.loc['USE3',['Coefficient', 'Standard error']],
        -ols_params.loc['MET2',['Coefficient', 'Standard error']],
        -ols_params.loc['ANA2',['Coefficient', 'Standard error']],
        2*ols_params.loc['PRICE',['Coefficient', 'Standard error']],
        3*ols_params.loc['PRICE',['Coefficient', 'Standard error']]
        ]
    add_params = pd.DataFrame(add_params, columns=['Coefficient', 'Standard error'],
                              index=['BIO1','SCA1','KNO1','USE1','MET1','ANA1','PRICE_30','PRICE_200'])
    ols_params_2 = pd.concat([ols_params_2, add_params], axis=0)
    ols_params_2.rename(index={'PRICE':'PRICE_03'}, inplace=True)
    ols_params_2 = ols_params_2.loc[['BIO1','BIO2','BIO3',
                                     'SCA1','SCA2','SCA3',
                                     'KNO1','KNO2','KNO3',
                                     'USE1','USE2','USE3',
                                     'MET1','MET2',
                                     'ANA1','ANA2',
                                     'PRICE_03','PRICE_30','PRICE_200',
                                     'Intercept'],:]
    
    return ols_params, ols_params_2

In [5]:
total_ols, total_ols_2 = get_ols_params_table(DATA)

# Directory
folder_root = 'C:/Users/mazy4/Dropbox/6. C&S Lab/7. 2024 Project/4. 야생생물 경제가치_생물자원관/2. 분석/5. Result/2. 본설문_Result/2. 천연물/HB'
if not os.path.isdir(folder_root):
    os.makedirs(folder_root)

# Data Frame -> csv file
total_ols_file_name = folder_root + '/' + '1_Naturalproducts_TOTAL_OLS.csv'
total_ols.to_csv(total_ols_file_name, mode='w')

# Result Check
total_ols

,Coefficient,Standard error,P-value
Intercept,5.015199,0.327007,0.000000
BIO2,0.581761,0.200250,0.003756
BIO3,0.657233,0.200250,0.001068
SCA2,0.128931,0.200250,0.519830
SCA3,0.628931,0.200250,0.001738
KNO2,0.169811,0.200250,0.396655
KNO3,0.191824,0.200250,0.338348
USE2,0.015723,0.200250,0.937432
USE3,0.157233,0.200250,0.432544
MET2,0.696541,0.173422,0.000064


---

#### **3. 개별 응답자의 부분 가치**

In [6]:
X = DATA[['BIO2','BIO3',
          'SCA2','SCA3',
          'KNO2','KNO3',
          'USE2','USE3',
          'MET2',
          'ANA2',
          'PRICE']].to_numpy().astype(np.float64)
y = DATA['Y'].to_numpy().astype(np.float64)

print(X.shape)
print(y.shape)

(954, 11)
(954,)


In [7]:
K = len(DATA['INDEX'].unique()) # 그룹 개수 : 개별 응답자 수
G = np.array([i for i in range(53) for _ in range(18)])

print(K)
print(G)

53
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 17 17 17 17 17 17
 17 17 17 17 17 17 17 17 17 17 17 17 18 18 18 18

In [8]:
with pm.Model() as HLM:
    
    # Prior probability (proposal distribution) - level2
    mu_a = pm.Normal('mu_1', mu=0, sigma=100)
    sigma_a = pm.HalfCauchy('sigma_1', 5)
    mu_b = pm.Normal('mu_2', mu=0, sigma=100)
    sigma_b = pm.HalfCauchy('sigma_2', 5)

    # Prior probability (proposal distribution)
    a = [pm.Normal(f'theta_Q{i}', mu=mu_a, sigma=sigma_a, shape=K) for i in range(1, X.shape[1]+1)]
    b = pm.Normal('intercept', mu=mu_b, sigma=sigma_b, shape=K)
    eps = pm.HalfCauchy('eps', 5)

    # Model
    y_est = b[G]
    for i in range(len(a)):
        y_est = y_est + a[i][G]*X[:,i]
    likelihood = pm.Normal('y', mu=y_est, sigma=eps, observed=y)

In [9]:
with HLM:
    HLM_trace = pm.sample(10000, progressbar=True, chains=10, cores=12, random_seed=123, target_accept=0.999)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (10 chains in 10 jobs)
NUTS: [mu_1, sigma_1, mu_2, sigma_2, theta_Q1, theta_Q2, theta_Q3, theta_Q4, theta_Q5, theta_Q6, theta_Q7, theta_Q8, theta_Q9, theta_Q10, theta_Q11, intercept, eps]


Output()

Sampling 10 chains for 1_000 tune and 10_000 draw iterations (10_000 + 100_000 draws total) took 1167 seconds.


In [11]:
person_index = DATA['INDEX'].unique()
person_index.sort()

personal_params = az.summary(HLM_trace).iloc[1:,[0]]
personal_params['params'] = personal_params.index.str.replace('\[[0-9]{,3}\]', '', regex=True)

# mu, sigma 제거
personal_params = personal_params[~personal_params['params'].isin(['mu_1', 'mu_2', 'sigma_1', 'sigma_2'])].copy()
personal_params['index'] = [
    i for param in personal_params['params'].unique() 
    if param not in ['mu_1', 'mu_2', 'sigma_1', 'sigma_2']
    for i in person_index
]

personal_params = personal_params.pivot(columns='params', index='index', values='mean').reset_index()
personal_params.columns = ['INDEX',
                           'Intercept',
                           'BIO2','BIO3',
                           'SCA2','SCA3',
                           'KNO2','KNO3',
                           'USE2','USE3',
                           'MET2',
                           'ANA2',
                           'PRICE']

# Output: csv file 저장
personal_params_file_name = folder_root + '/' + '2_Naturalproducts_HBM_Personal_params.csv'
personal_params.to_csv(personal_params_file_name, mode='w', index=False)

# Result 확인
personal_params

,INDEX,Intercept,BIO2,BIO3,SCA2,SCA3,KNO2,KNO3,USE2,USE3,MET2,ANA2,PRICE
0,14,5.215,0.582,-0.067,-1.543,0.580,0.637,0.756,0.539,-0.053,0.078,-0.039,0.091
1,17,10.243,-0.033,-0.215,-1.941,0.086,-0.494,0.097,0.062,0.892,0.419,0.538,1.028
2,19,1.466,0.915,0.564,-0.275,1.629,-0.662,0.047,0.949,0.240,0.765,0.648,2.118
3,22,1.594,1.663,0.359,-0.479,1.187,-0.511,0.199,0.824,-0.003,0.306,-0.168,0.360
4,23,2.610,0.528,2.339,-0.585,0.173,1.290,2.119,-0.729,-0.376,-0.493,-0.611,-0.302
5,24,4.814,0.093,0.914,0.036,-0.143,0.316,0.082,0.081,0.319,-0.147,0.092,1.070
6,25,1.788,2.247,1.656,-0.472,1.890,-0.012,-0.368,0.741,-0.442,0.265,0.029,0.414
7,26,3.856,0.292,2.407,-0.823,-0.301,-0.061,0.054,0.571,0.098,0.226,-0.011,1.628
8,27,4.868,0.112,1.329,-1.762,0.587,0.071,2.439,0.071,-0.047,-0.585,0.835,3.659
9,28,0.548,0.012,-0.067,-0.096,0.247,-0.042,0.072,-0.097,-0.097,0.183,0.299,2.419


---

#### **4. Clustering**

In [12]:
kmeans = KMeans(n_clusters=2, random_state=456)
clusters = kmeans.fit(personal_params.iloc[:,1:])

personal_params['Cluster'] = clusters.labels_
DATA_cluster = pd.merge(left=DATA,
                        right=personal_params[['INDEX','Cluster']],
                        how='left',
                        on='INDEX')

# Output : csv file
data_cluster_file_name = folder_root + '/' + '3_Naturalproducts_OLS_Data_Cluster.csv'
DATA_cluster.to_csv(data_cluster_file_name, mode='w', index=False)

# Result Check
DATA_cluster

,INDEX,CARD,BIO1,BIO2,BIO3,SCA1,SCA2,SCA3,KNO1,KNO2,...,USE1,USE2,USE3,MET1,MET2,ANA1,ANA2,PRICE,Y,Cluster
0,14,1,1,0,0,0,0,1,0,1,...,0,1,0,1,0,1,0,3,2,0
1,14,2,0,1,0,0,1,0,1,0,...,0,1,0,1,0,0,1,2,3,0
2,14,3,0,1,0,1,0,0,0,1,...,0,0,1,0,1,0,1,1,5,0
3,14,4,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,1,3,1,0
4,14,5,0,1,0,0,0,1,0,1,...,1,0,0,0,1,1,0,2,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949,134,14,0,0,1,1,0,0,0,1,...,0,1,0,1,0,0,1,3,0,0
950,134,15,0,0,1,0,1,0,0,1,...,1,0,0,1,0,1,0,2,0,0
951,134,16,0,1,0,1,0,0,1,0,...,1,0,0,1,0,1,0,3,0,0
952,134,17,0,0,1,0,0,1,1,0,...,0,0,1,0,1,1,0,3,2,0


##### 1st Group

In [13]:
idx_1 = (DATA_cluster['Cluster'] == 0)
group1_ols, group1_ols_2 = get_ols_params_table(DATA_cluster.loc[idx_1,:])
group1_ols

,Coefficient,Standard error,P-value
Intercept,3.155702,0.350155,0.000000
BIO2,0.789474,0.214425,0.000250
BIO3,0.842105,0.214425,0.000095
SCA2,0.236842,0.214425,0.269752
SCA3,0.842105,0.214425,0.000095
KNO2,0.157895,0.214425,0.461767
KNO3,0.078947,0.214425,0.712854
USE2,0.026316,0.214425,0.902360
USE3,0.197368,0.214425,0.357666
MET2,0.901316,0.185698,0.000002


##### 2nd Group

In [14]:
idx_2 = (DATA_cluster['Cluster'] == 1)
group2_ols, group2_ols_2 = get_ols_params_table(DATA_cluster.loc[idx_2,:])
group2_ols

,Coefficient,Standard error,P-value
Intercept,9.725926,0.559243,0.000000
BIO2,0.055556,0.342465,0.871258
BIO3,0.188889,0.342465,0.581729
SCA2,-0.144444,0.342465,0.673538
SCA3,0.088889,0.342465,0.795413
KNO2,0.200000,0.342465,0.559730
KNO3,0.477778,0.342465,0.164181
USE2,-0.011111,0.342465,0.974143
USE3,0.055556,0.342465,0.871258
MET2,0.177778,0.296583,0.549419


---

#### **5. Result Clean-Up**

In [15]:
summary_coef = pd.concat([total_ols_2, group1_ols_2, group2_ols_2], axis=1)
summary_coef.columns = ['Full sample', 'Full sample SE', 'Cluster 1', 'Cluster 1 SE', 'Cluster 2', 'Cluster 2 SE']
add_lines = pd.DataFrame([[len(DATA['INDEX'].unique()),
                           len(DATA_cluster.loc[idx_1,'INDEX'].unique()),
                           len(DATA_cluster.loc[idx_2,'INDEX'].unique())]],
                         columns=['Full sample', 'Cluster 1', 'Cluster 2'], index=['Number of cases'])
summary_coef = pd.concat([summary_coef, add_lines], axis=0)

# Output : csv file
summary_coef_file_name = folder_root + '/' + '4_Naturalproducts_Summary_Coef.csv'
summary_coef.to_csv(summary_coef_file_name, mode='w')

# Result Check
summary_coef

,Full sample,Full sample SE,Cluster 1,Cluster 1 SE,Cluster 2,Cluster 2 SE
BIO1,-1.238994,-0.400500,-1.631579,-0.428850,-0.244444,-0.684930
BIO2,0.581761,0.200250,0.789474,0.214425,0.055556,0.342465
BIO3,0.657233,0.200250,0.842105,0.214425,0.188889,0.342465
SCA1,-0.757862,-0.400500,-1.078947,-0.428850,0.055556,-0.684930
SCA2,0.128931,0.200250,0.236842,0.214425,-0.144444,0.342465
SCA3,0.628931,0.200250,0.842105,0.214425,0.088889,0.342465
KNO1,-0.361635,-0.400500,-0.236842,-0.428850,-0.677778,-0.684930
KNO2,0.169811,0.200250,0.157895,0.214425,0.200000,0.342465
KNO3,0.191824,0.200250,0.078947,0.214425,0.477778,0.342465
USE1,-0.172956,-0.400500,-0.223684,-0.428850,-0.044444,-0.684930
